# Мова як послідовність

## I. Run-on Sentences

### 1. Домен

Цього тижня ви працюватимете над задачею виправлення помилок.

Run-on речення - це речення, склеєне з двох чи більше речень без належної пунктуації. Таку помилку часто допускають механічно, коли швидко друкують текст, проте така помилка виникає і від незнання мови. Особливо часто ця помилка зустрічається в інтернет-спілкуванні.

Наприклад:
```
Thanks for talking to me let's meet again tomorrow Bye.
```

У цьому реченні насправді три склеєні речення. Правильний варіант:
```
Thanks for talking to me. Let's meet again tomorrow. Bye.
```

Run-on речення важливо визначати не лише для виправлення помилок. Ця помилка впливає на якість визначення сутностей, частин мови, синтаксичних зв'язків тощо.

Більше інформації та прикладів можна знайти за посиланнями:
- http://www.bristol.ac.uk/arts/exercises/grammar/grammar_tutorial/page_37.htm
- https://www.english-grammar-revolution.com/run-on-sentence.html
- https://www.quickanddirtytips.com/education/grammar/what-are-run-on-sentences

### 2. Класифікатор

Дані:
- Виберіть будь-який відкритий корпус та згенеруйте тренувальні дані для моделі. Тренувальними даними буде набір склеєних речень. Візьміть до уваги, що склеєних речень може бути кілька (зазвичай 2, але буває і 3-4), а перше слово наступного речення може писатися з великої чи малої літери.
- Зберіть (чи знайдіть у відкритому доступі) базу енграмів. Візьміть до уваги, що відкриті бази енграмів зазвичай містять статистику, зібрану на реченнях, а отже вони можуть не містити енграми на межі речень.

Тестування:
- Напишіть бейзлайн та метрику для тестування якості.
- Для тестування використайте корпус [run-on-test.json](run-on-test.json). Формат корпусу:
```
[
  [
    ["Thanks", false],
    ["for", false],
    ["talking", false],
    ["to", false],
    ["me", true],
    ["let", false],
    ["'s", false],
    ["meet", false],
    ["again", false],
    ["tomorrow", true],
    ["Bye", false],
    [".", false]
  ],
...
]
```

`true` позначає слово, на якому закінчується речення. Тестовий корпус містить 200 речень (~ 4700 токенів). 3% токенів мають клас `true`, а решта - `false`.

Класифікатор:
- Виділіть ознаки, які впливають на те, чи є слово на межі речень. Подумайте про правий/лівий контекст, написання слова, граматичні ознаки (чи може речення закінчитись на сполучник?), енграми (чи часто це слово і наступне йдуть поруч?), складники та залежності тощо.
- Побудуйте класифікатор на основі логістичної регресії з використанням виділених ознак, який анотує послідовно слова у реченні на предмет закінчення речення.
- Спробуйте покращити якість роботи класифікатора, змінюючи набір чи комбінацію ознак.

In [1]:
import spacy
import random
import string
import pickle
import spacy
import pandas as pd

from spacy.tokens import Doc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

pd.set_option('display.max_colwidth', 400)

## Read data
1. Dataset of simplified text from wikipedia (for training)

Знайшла датасет документів з Simple English Wikipedia, який був створений для Text simplification.<br>
Я взяла тільки один файл зі спрощеними реченнями - simple.txt

Посилання - http://www.cs.pomona.edu/~dkauchak/simplification/

2. Frequent trigrams (for extra feature)

Coca 3-grams for words which occur at least 10 times

Посилання - https://www.wordfrequency.info/ngrams.asp

#### 1. Dataset for training

In [2]:
nlp = spacy.load("en", disable=['tagger', 'parser', 'ner'])

def get_data(file='Data/document-aligned.v2/simple.txt'):
    with open(file) as f:
        prev_subj = 'April'
        prev_num = '0'
        subject = ''
        num = '0'
        sentence = ''
        paragraphs = []
        paragraph = []

        for line in f.readlines():
            elements = line.rstrip('\n').split('\t')
            subject = elements[0]
            num = elements[1]
            sentence = elements[2]
            tokens = nlp(sentence)
            
            if subject != prev_subj and num != prev_num and paragraph:
                paragraph[-1][1] = False
                paragraph.append(['</p>', False])
                
                paragraphs.append(paragraph)
                paragraph = []
            
            for i, token in enumerate(tokens[:-1]):  # dot is last character
                if not paragraph:
                    paragraph.append(['<p>', False])

                if i == 0 and len(paragraph) > 1:  # first word of the sentence
                    # randomly make a word lower- or upper-case
                    random_num = random.randint(0,1)
                    if random_num == 1:
                        current_token =  token.text.lower()
                    else:
                        current_token =  token.text.title()
                    paragraph.append([current_token, False])
                elif i == len(tokens) - 2:
                    paragraph.append([token.text, True])
                else:
                    paragraph.append([token.text, False])
            
            prev_subj = subject
            prev_num = num
        return paragraphs
    
# paragraphs = get_data()

In [3]:
def add_to_pickle(path, item):
    with open(path, 'ab') as file:
        pickle.dump(item, file, pickle.HIGHEST_PROTOCOL)


def read_from_pickle(path):
    objects = []
    with (open(path, "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break
    return objects


# add_to_pickle('Data/procc_dataset.pkl', paragraphs)
paragraphs = read_from_pickle('Data/procc_dataset.pkl')[0][0]
paragraphs[0][:10]

[['<p>', 0],
 ['April', 0],
 ['is', 0],
 ['the', 0],
 ['fourth', 0],
 ['month', 0],
 ['of', 0],
 ['the', 0],
 ['year', 1],
 ['it', 0]]

#### 2. Frequent N-grams Dataset

In [4]:
def get_freq_trigrams():
    with open('Data/ngrams_alpha.txt', 'r') as f:
        lines = f.read().split('\n')
        lines_list = [line.split('\t') for line in lines]
        df = pd.DataFrame(lines_list, columns=['freq', 'w1', 'w2', 'w3'])
        df['freq'] = pd.to_numeric(df['freq'])
        df['w3'] = df['w3'].map(lambda x: str(x).lower())
        df = df[df.apply(lambda x: True if (x['w2']=='.') else False, axis=1)]
        df['trigrams_dict'] = df.apply(lambda x: {str(x['w1']) + ' ' +str(x['w3']): x['freq']}, axis=1)
    
    trigrams_list = df['trigrams_dict'].tolist()
    trigrams_dict = {key: value for item in trigrams_list for key, value in item.items()}

    return trigrams_dict

# df.sort_values(by='freq', ascending=False).head(10)
# 14594 lines with dot as second word

trigrams_dict  = get_freq_trigrams()
list(trigrams_dict.items())[:10]

[('free <p>', 564.0),
 ('free "', 501.0),
 ('free the', 225.0),
 ('free i', 159.0),
 ('free but', 129.0),
 ('free and', 125.0),
 ('free call', 115.0),
 ('free it', 111.0),
 ('free he', 91.0),
 ('free they', 70.0)]

In [5]:
a = max(trigrams_dict.values())
b = min(trigrams_dict.values())

c = 0
d = 10

def transform_num(x, a=a, b=b, c=c, d=d):
    if x:
        y = (x - a) * (d - c)/(b - a) + c
        return round(y)
    else:
        return None

transform_num(1000)

8

Функція ставить у відповідність числу з [10, ..., 5905] -> число в [0, ..., 10]. Таким чином маємо ознаку, яка має 11 можливих значень: 0 - крапка між даними словами вживається рідко, .., 10 - вживається часто.

## Training

In [7]:
def prepair_X_y(paragraphs):
    X = []
    y = []
    for p in paragraphs:
        X.append([word for word, _ in p])
        y.append([boo for _, boo in p])
    return X, y

In [8]:
X, y = prepair_X_y(paragraphs)

X_train_, X_test_, y_train_, y_test_ = train_test_split(X, y, test_size=0.33, random_state=42)

X_train_ = X_train_[:5000]
X_test_ = X_test_[:1000]
y_train = [token for x in y_train_[:5000] for token in x]
y_test =[token for x in y_test_[:1000] for token in x]

In [9]:
class WordTokenizer(object):
    """
    Custom Tokenizer
    """
    def __init__(self, vocab=nlp.vocab, tokenizer=None, return_doc=True):
        self.vocab = vocab
        self._word_tokenizer = tokenizer
        self.return_doc = return_doc

    def __call__(self, text):
        if self._word_tokenizer:
            words = self._word_tokenizer.tokenize(text)
        else:
            words = text.split(' ')
        if self.return_doc:
            spaces = [True] * len(words)
            return Doc(self.vocab, words=words, spaces=spaces)
        else:
            return words

nlp = spacy.load('en', disable=['ner'])
nlp.tokenizer = WordTokenizer(nlp.vocab)

In [27]:
def processed(docs):
    
    tokens = []
    for doc in docs:
        doc_tokens = [(token.text, token.lemma_, token.pos_, token.dep_) for token in nlp(" ".join(doc))]
        tokens.extend(doc_tokens)
    
    df = pd.DataFrame(tokens, columns=['token', 'lemma', 'pos', 'dep'])
    df['capital'] = df['token'].map(lambda t: 1 if t.istitle() else 0)
    df['paragraph_boundary'] = df['token'].map(lambda t: 1 if t in ('<p>', '</p>') else 0)
    df['number'] = df['token'].map(lambda t: 1 if t.isnumeric() else 0)
    df['dot'] = df['token'].map(lambda t: 1 if t == '.' else 0)
    df['is_punct'] = df['token'].map(lambda t: 1 if str(t) in string.punctuation else 0)
    
    df['prev_lemma'] = df['lemma'].shift(1)
    df['prev_token_pos'] = df['pos'].shift(1)
    df['prev_dep'] = df['dep'].shift(1)
    df['prev_paragraph_boundary'] = df['token'].shift(1).map(lambda t: 1 if t in ('<p>', '</p>') else 0)
    df['prev_capital'] = df['token'].shift(1).map(lambda t: 1 if str(t).istitle() else 0)
    
    df['next_lemma'] = df['lemma'].shift(-1)
    df['next_token_pos'] = df['pos'].shift(-1)
    df['next_dep'] = df['dep'].shift(-1)
    df['next_paragraph_boundary'] = df['token'].shift(-1).map(lambda t: 1 if t in ('<p>', '</p>') else 0)
    df['next_capital'] = df['token'].shift(-1).map(lambda t: 1 if str(t).istitle() else 0)
    
    df['neighbors'] = df['token'].shift(1) + ' ' + df['token'].shift(-1)
    
    # Feature get frequent trigams list
    df['bigram'] = df['token'] + ' ' + df['token'].shift(-1)
    df['bigram_group'] = df['bigram'].map(lambda x: transform_num(trigrams_dict.get(x)))
    
    return df.fillna('').drop('token', axis=1)

processed(X_train_).head(10)

,lemma,pos,dep,capital,paragraph_boundary,number,dot,is_punct,prev_lemma,prev_token_pos,...,prev_paragraph_boundary,prev_capital,next_lemma,next_token_pos,next_dep,next_paragraph_boundary,next_capital,neighbors,bigram,bigram_group
0,<p>,PUNCT,ROOT,0,1,0,0,0,,,...,0,0,the,DET,det,0,1,,<p> The,
1,the,DET,det,1,0,0,0,0,<p>,PUNCT,...,1,0,Santa,PROPN,compound,0,1,<p> Santa,The Santa,
2,Santa,PROPN,compound,1,0,0,0,0,the,DET,...,0,1,Ana,PROPN,compound,0,1,The Ana,Santa Ana,
3,Ana,PROPN,compound,1,0,0,0,0,Santa,PROPN,...,0,1,River,PROPN,nsubj,0,1,Santa River,Ana River,
4,River,PROPN,nsubj,1,0,0,0,0,Ana,PROPN,...,0,1,be,VERB,ccomp,0,0,Ana is,River is,
5,be,VERB,ccomp,0,0,0,0,0,River,PROPN,...,0,1,a,DET,det,0,0,River a,is a,
6,a,DET,det,0,0,0,0,0,be,VERB,...,0,0,major,ADJ,amod,0,0,is major,a major,
7,major,ADJ,amod,0,0,0,0,0,a,DET,...,0,0,river,NOUN,attr,0,0,a river,major river,
8,river,NOUN,attr,0,0,0,0,0,major,ADJ,...,0,0,in,ADP,prep,0,0,major in,river in,
9,in,ADP,prep,0,0,0,0,0,river,NOUN,...,0,0,southern,ADJ,amod,0,0,river southern,in southern,


In [28]:
vectorizer = DictVectorizer()

X = processed(X_train_)
X_train = vectorizer.fit_transform(X.to_dict('records'))

X = processed(X_test_)
X_test = vectorizer.transform(X.to_dict('records'))

## Train, Test

In [29]:
lrc = LogisticRegression(random_state=42, solver="lbfgs", max_iter=1000)

lrc.fit(X_train, y_train)
y_pred = lrc.predict(X_test)

print(classification_report(y_true=y_test, y_pred=y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.981     0.995     0.988    345759
           1      0.880     0.666     0.758     19470

   micro avg      0.977     0.977     0.977    365229
   macro avg      0.931     0.831     0.873    365229
weighted avg      0.976     0.977     0.976    365229



## Validation

In [30]:
import json
with open('Data/run-on-test.json') as file:
    data = json.load(file)
    data = [[['<p>', False]] + i + [['</p>', False]] for i in data]
    
X_val_, y_val = prepair_X_y(data)
y_val =[token for x in y_val for token in x]
X = processed(X_val_)
X_val = vectorizer.transform(X.to_dict('records'))

y_pred = lrc.predict(X_val)

print(classification_report(y_true=y_val, y_pred=y_pred, digits=3))

              precision    recall  f1-score   support

       False      0.993     0.992     0.992      4942
        True      0.744     0.787     0.765       155

   micro avg      0.985     0.985     0.985      5097
   macro avg      0.869     0.889     0.879      5097
weighted avg      0.986     0.985     0.985      5097



Побудувана модель логістичної регресії з такими кінцевими ознаками:
1. lemma
2. pos
3. dep
4. capital
5. paragraph_boundary
6. number
7. dot
8. is_punct
9. prev_lemma
10. prev_token_pos
11. prev_dep
12. prev_paragraph_boundary
13. prev_capital
14. next_lemma
15. next_token_pos
16. next_paragraph_boundary
17. next_capital
18. neighbors
18. bigram
19. bigram_group

Класифікувати виключно на лемах дало кращу якість. Для класифікації чи після даного слова треба ставити крапку взяла ознаки попереднього, поточного і наступного слів. Перевірила чи покращиться модель, якщо взяти 2ге слово зліва, якість не покращилась. І додатково знайшла датасет частих триграм, з них використала тільки ті триграми, які мають 2им елементом крапку і на цьому базуючись зробила ознаку bigram_group.